In [1]:
import os
import sys
import json

from os.path import join as ospj
from os.path import expanduser
from munch import Munch as mch
import numpy as np

from ds import prepare_coco_dataloaders, prepare_flickr_dataloaders, prepare_cub_dataloaders, prepare_flo_dataloaders, build_vocab

from utils import *
from networks import *
from train_probVLM import *

import matplotlib.pyplot as plt

/vol/tensusers4/nhollain/ProbVLM/src/train_probVLM.py:32: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:83.)
  def train_ProbVLM(CLIP_Net, BayesCap_Net, train_loader, eval_loader, Cri = TempCombLoss(), device='cuda', dtype=torch.cuda.FloatTensor(),


In [2]:
from probvlm_data_loader import get_data as probvlm_get_data

In [3]:
# vocab = build_vocab('/vol/tensusers5/nhollain/ProbVLM/', data_name = 'coco', 
#                     jsons = {'coco': ['captions_train2014.json', 'captions_val2014.json']}, threshold=0)
# vocab = build_vocab('/vol/tensusers5/nhollain/ProbVLM/coco', threshold = 0)
# vocab

# import pickle
# with open('/vol/tensusers5/nhollain/ProbVLM/src/ds/vocabs/coco_vocab.pkl', 'wb') as f:
#     pickle.dump(vocab, f, pickle.HIGHEST_PROTOCOL)

In [4]:
import itertools
sys.path.append('/vol/tensusers4/nhollain/thesis2023-2024/s_clip_scripts')
# from data_loader import get_data
from params import parse_args
from open_clip import create_model_and_transforms, get_tokenizer, create_loss

def prep_str_args(str_args): # Code to parse the string style arguments, as shown below
    str_args = str_args.split('\n') # Split on newline
    str_args = [s.strip() for s in str_args] # Remove any whitespaces from the start and end of the strings
    # Split on the space between the parameter name and the value, e.g. '--name x' becomes ['--name', 'x']
    str_args = [s.split(' ') for s in str_args] 
    str_args = list(itertools.chain(*str_args)) # Flatten the resulting list of lists
    str_args = [s for s in str_args if len(s) > 0] # Remove arguments that are empty
    return str_args

str_args = ''' 
    --train-data RS-ALL
    --val-data RS-ALL
    --imagenet-val RSICD-CLS \
    '''

str_args = prep_str_args(str_args)
args = parse_args(str_args)


model, preprocess_train, preprocess_val = create_model_and_transforms(
        args.model, args.pretrained, precision=args.precision, device=args.device, output_dict=True,
        aug_cfg = args.aug_cfg, )

In [5]:
probvlm_data = probvlm_get_data((preprocess_train, preprocess_val), tokenizer=get_tokenizer(args.model), model = model, batch_size = 1024)

Coco data (split: val)	Coco data (split: train)	

In [6]:
# data = get_data(args, (preprocess_train, preprocess_val), iter=0, tokenizer=get_tokenizer(args.model), model = model)

In [7]:
cub_train_loader, cub_valid_loader = probvlm_data['train'].dataloader, probvlm_data['val'].dataloader

In [8]:
# rs_train_loader, rs_valid_loader = data['train'].dataloader, data['val'].dataloader

In [9]:
# dataset = 'coco' # CUB' # coco or flickr
# data_dir = ospj('/vol/tensusers5/nhollain/ProbVLM/', dataset) # e.g. ospj(expanduser('~'), 'Documents', 'jm', 'data', dataset)
# dataloader_config = mch({
#     'batch_size': 64,
#     'random_erasing_prob': 0.,
#     'traindata_shuffle': True
# })
# loaders,vocab = load_data_loader(dataset, data_dir, dataloader_config)
# cub_train_loader, cub_valid_loader, cub_test_loader = loaders['train'], loaders['val'], loaders['test']

In [10]:
# clip_net = load_model('cuda')
CLIP_Net = load_model(device='cuda', model_path=None)
ProbVLM_Net = BayesCap_for_CLIP(inp_dim=512, out_dim=512, hid_dim=256, num_layers=3, p_drop=0.05,)

In [ ]:
train_ProbVLM(CLIP_Net, ProbVLM_Net, cub_train_loader, cub_valid_loader, Cri = TempCombLoss(), device='cuda', dtype=torch.cuda.FloatTensor,
    init_lr=8e-5, num_epochs=10, eval_every=5, ckpt_path='../ckpt/ProbVLM_Net', T1=1e0, T2=1e-4,)

=> no checkpoint found at ''


Epoch 0:   5%|██████▍                                                                                                                         | 4/80 [03:01<52:26, 41.40s/batch, loss=10.1]